## Reading Data

In [11]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

#increase width to 100%
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.optimizers import SGD
import seaborn as sns

#add fastai path
homedir = os.path.expanduser('~')
sys.path.append(os.path.join(homedir,'fastai'))
from fastai.structured import add_datepart

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cross_validation import train_test_split

#### List, read the files and display summaries

In [3]:
datadir = Path(os.path.expanduser('~')).joinpath('data/kaggle/rossmann-store-sales')
files = [o for o in datadir.iterdir() if str(o).endswith('.csv')]
files

[PosixPath('/home/achinta/data/kaggle/rossmann-store-sales/train.csv'),
 PosixPath('/home/achinta/data/kaggle/rossmann-store-sales/test.csv'),
 PosixPath('/home/achinta/data/kaggle/rossmann-store-sales/store.csv'),
 PosixPath('/home/achinta/data/kaggle/rossmann-store-sales/sample_submission.csv')]

In [4]:
def read_data():
#     filenames = ['train','test','store']
#     train_df, test_df, store_df = [pd.read_csv(datadir.joinpath(filename + '.csv'),low_memory=False) for filename in filenames]
    train_df = pd.read_csv(datadir.joinpath('train.csv'),parse_dates=['Date'],low_memory=False)
    test_df = pd.read_csv(datadir.joinpath('test.csv'),parse_dates=['Date'],low_memory=False)
    store_df = pd.read_csv(datadir.joinpath('store.csv'))
    return train_df, test_df, store_df

train_df, test_df, store_df = read_data()

In [5]:
from IPython.display import HTML
for df in [train_df, test_df, store_df]:
    display(df.head(3))
    

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [6]:
for df in [train_df, test_df, store_df]:
    display(df.describe())

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000


,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


## Understanding the data

## Adding Features

In [28]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

#create scalers for continuous variables
def scalers_fit(df, cols):
    scalers = {}
    for col in cols:
        scalers[col] = MinMaxScaler().fit(df[col])
    return scalers

#transform the continuous variable columns for which scalers are available
def scalers_transform(df, scalers={},cols=[]):
    #if no columns are passed, transform all columns for which scalers are available
    if not cols:
        cols = scalers.keys()
    for col in scalers:
        df[col] = scaler.transform(df[col])
    return df

#create label encoders for categorical variables
def encoders_fit(df, cols):
    encoders = {}
    for col in cols:
        encoders[col] = LabelEncoder().fit(df[col])
    return encoders

#transform teh categorical variables using previously created encoders
def encoders_transform(df, encoders={},cols=[]):
    if not cols:
        cols = encoders.keys()
    for col in cols:
        df[col] = encoders[col].transform(df[col])
    return df

In [21]:
def add_features(train_df, test_df, store_df):
    add_datepart(train_df, "Date", drop=False)
    add_datepart(test_df, "Date", drop=False)
    
    #convert stateHoliday to boolean
    train_df.StateHoliday = train_df.StateHoliday!='0'
    test_df.StateHoliday = test_df.StateHoliday!='0'
    
    #Join store
    train_df = join_df(train_df, store_df, "Store")
    test_df = join_df(test_df, store_df, "Store")
    
    #remove the duplicate joined columns
    for df in [train_df, test_df]:
        for c in df.columns:
            if c.endswith('_y'):
                if c in df.columns:
                    df.drop(c, inplace=True, axis=1)
                    
    return (train_df, test_df)

train_df, test_df, store_df = read_data()
train_df, test_df = add_features(train_df, test_df, store_df)

In [9]:
print(train_df.shape)
print(test_df.shape)
print('train columns not present in test: ',[o for o in train_df.columns if o not in test_df.columns])
print('test columns not present in train: ',[o for o in test_df.columns if o not in train_df.columns])

(1017209, 31)
(41088, 30)
train columns not present in test:  ['Sales', 'Customers']
test columns not present in train:  ['Id']


### Define categorical and numeric columns

In [29]:
date_cols = ['Year','Month','Week','Day','Dayofweek','Dayofyear','Is_month_end','Is_month_start','Is_quarter_end']
date_cols += ['Is_quarter_end','Is_quarter_start','Is_year_end','Is_year_start',]

cat_vars = date_cols + ['Store','StoreType','Assortment','Promo2']
# cat_vars += ['PromoInterval']

contin_vars = ['CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear']

### scale numeric columns
# scalers = scalers_fit(train_df,contin_vars)
# train_df = scalers_transform(train_df, scalers)

#encode categorical columns
encoders = encoders_fit(train_df, cat_vars)
train_df = encoders_transform(train_df, encoders)

### embeddings for categorical columns


# train_df.columns

encoding... Year
encoding... Month
encoding... Week
encoding... Day
encoding... Dayofweek
encoding... Dayofyear
encoding... Is_month_end
encoding... Is_month_start
encoding... Is_quarter_end
encoding... Is_quarter_end
encoding... Is_quarter_start
encoding... Is_year_end
encoding... Is_year_start
encoding... Store
encoding... StoreType
encoding... Assortment
encoding... Promo2


In [ ]:
def get_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape=))